In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy.ndimage import uniform_filter
from scipy.stats import gamma as gamma_dist
from pathlib import Path
from tqdm.notebook import tqdm
from glob import glob
import sys
import gc
import psutil

project_root = Path.cwd().parents[1] # Up from learningnotebooks/phase4_sar_codec/
print(project_root)
sys.path.insert(0, str(project_root / "src"))

from utils.io import load_sar_image, find_all_sar_files, get_info
from data.preprocessing import preprocess_sar_complete, extract_patches


rawdir = project_root/"data"/"raw"
outputdir = project_root/"data"/"patches"
checkpointdir = project_root/"checkpoints"
patchsize = 256
stride = 128

D:\Projects\CNNAutoencoderProject


In [2]:
sar_files = find_all_sar_files(rawdir)

In [3]:
safe_folders = list(rawdir.glob("*.SAFE"))
print(f"Total .SAFE folders: {len(safe_folders)}\n")

Total .SAFE folders: 22



In [4]:
complete = []
incomplete = []

for folder in safe_folders:
    measurement_dir = folder / "measurement"
    if measurement_dir.exists():
        tiffs = list(measurement_dir.glob("*.tiff"))
        if len(tiffs) >= 2:
            complete.append(folder.name)
        else:
            incomplete.append((folder.name, len(tiffs)))
    else:
        incomplete.append((folder.name, "no measurement folder"))

print(f"Complete: {len(complete)}")
print(f"Incomplete: {len(incomplete)}\n")

if incomplete:
    print("Incomplete folders:")
    for name, issue in incomplete:
        print(f"  {name}: {issue}")

Complete: 22
Incomplete: 0



In [5]:
folder = rawdir / "S1A_IW_GRDH_1SDV_20260117T061452_20260117T061517_062803_07E08D_824A.SAFE"

print(f"Folder exists: {folder.exists()}")
print(f"\nContents:")
for item in folder.iterdir():
    print(f"  {item.name}")
    
# Check if measurement exists with different casing
print(f"\nMeasurement folder check:")
print(f"  'measurement' exists: {(folder / 'measurement').exists()}")
print(f"  'MEASUREMENT' exists: {(folder / 'MEASUREMENT').exists()}")

Folder exists: True

Contents:
  annotation
  manifest.safe
  measurement
  preview
  S1A_IW_GRDH_1SDV_20260117T061452_20260117T061517_062803_07E08D_824A.SAFE-report-20260117T070206.pdf
  support

Measurement folder check:
  'measurement' exists: True
  'MEASUREMENT' exists: True


In [6]:
print(f"Found {len(sar_files)} TIFF files:\n")
for f in sar_files:
    info = get_info(f)
    print(f"  {info['satellite'].upper()} | {info.get('date', '?')} | {Path(f).name[:50]}...")

Found 46 TIFF files:

  S1A | 20251114 | s1a-iw-grd-vh-20251114t225109-20251114t225134-0618...
  S1A | 20251114 | s1a-iw-grd-vv-20251114t225109-20251114t225134-0618...
  S1A | 20260111 | s1a-iw-grd-vh-20260111t095455-20260111t095520-0627...
  S1A | 20260111 | s1a-iw-grd-vv-20260111t095455-20260111t095520-0627...
  S1A | 20260111 | s1a-iw-grd-vh-20260111t233937-20260111t234002-0627...
  S1A | 20260111 | s1a-iw-grd-vv-20260111t233937-20260111t234002-0627...
  S1A | 20260116 | s1a-iw-grd-vh-20260116t113541-20260116t113606-0627...
  S1A | 20260116 | s1a-iw-grd-vv-20260116t113541-20260116t113606-0627...
  S1A | 20260116 | s1a-iw-grd-vh-20260116t231538-20260116t231603-0627...
  S1A | 20260116 | s1a-iw-grd-vv-20260116t231538-20260116t231603-0627...
  S1A | 20260117 | s1a-iw-grd-vh-20260117t061452-20260117t061517-0628...
  S1A | 20260117 | s1a-iw-grd-vv-20260117t061452-20260117t061517-0628...
  S1A | 20260117 | s1a-iw-grd-vh-20260117t104141-20260117t104206-0628...
  S1A | 20260117 | s1a-iw-grd

In [7]:
path = sar_files[0]
import rasterio
import time
with rasterio.open(path) as src:
    print(f"Shape: {src.shape}")
    print(f"Dtype: {src.dtypes[0]}")
    image = src.read(1).astype(np.float32)
    print(f"Array size: {image.nbytes / 1e9:.2f} GB")

Shape: (16704, 25424)
Dtype: uint16
Array size: 1.70 GB


In [8]:
bounds_file = checkpointdir / "global_bounds.npy"

if bounds_file.exists():
    bounds = np.load(bounds_file, allow_pickle=True).item()
    global_vmin = bounds["vmin"]
    global_vmax = bounds["vmax"]
    print(f"Loaded bounds: [{global_vmin:.2f}, {global_vmax:.2f}] dB")
else:
    # Run the scanning loop to compute them
    print("No saved bounds, computing...")

    all_vmins = []
    all_vmaxs = []



    for path in tqdm(sar_files, desc="Scanning"):
        print(f"\n{path}")
        
        t0 = time.time()
        with rasterio.open(path) as src:
            image = src.read(1).astype(np.float32)
        print(f"  Read: {time.time() - t0:.1f}s")
        
        t0 = time.time()
        valid = image[image > 0]
        del image
        gc.collect()
        print(f"  Valid mask: {time.time() - t0:.1f}s")
        
        if len(valid) == 0:
            continue
        
        t0 = time.time()
        if len(valid) > 100_000:
            idx = np.random.randint(0, len(valid), 100_000)
            valid = valid[idx]
        print(f"  Subsample: {time.time() - t0:.1f}s")
        
        t0 = time.time()
        image_db = 10 * np.log10(np.maximum(valid, 1e-10))
        all_vmins.append(np.percentile(image_db, 1))
        all_vmaxs.append(np.percentile(image_db, 99))
        print(f"  Percentiles: {time.time() - t0:.1f}s")
        
        del valid, image_db
        gc.collect()

    global_vmin = np.median(all_vmins)
    global_vmax = np.median(all_vmaxs)


    print(f"\nGlobal bounds: [{global_vmin:.2f}, {global_vmax:.2f}] dB")
    np.save(checkpointdir / "global_bounds.npy", {
        "vmin" : global_vmin,
        "vmax" : global_vmax,
        "all_vmins" : all_vmins,
        "all_vmaxs" : all_vmaxs
    })

Loaded bounds: [14.77, 24.54] dB


In [10]:

min_valid = 0.9

all_patches = []
filestats = []

print("Pass 2 Extracting Patches...\n")

for path in tqdm(sar_files, desc = "Processing"):
    filename = Path(path).stem

    mem_gb = psutil.Process().memory_info().rss / 1e9
    if mem_gb > 25:  # bail out before crash
        print(f"WARNING: Memory at {mem_gb:.1f} GB, stopping early")
        break

    image = load_sar_image(path)
    image_shape=image.shape

    normalised, params = preprocess_sar_complete(
        image, 
        vmin=global_vmin, 
        vmax=global_vmax
    )
    del image
    gc.collect()
    
    # Extract patches
    patches, positions = extract_patches(
        normalised,
        patch_size=patchsize,
        stride=stride,
        min_valid=min_valid
    )
    del normalised
    gc.collect()
    
    n_patches = len(patches)
    filestats.append({
        'filename': filename,
        'shape': image_shape,
        'patches': n_patches
    })
    if n_patches > 0:
        np.save(outputdir / f"{filename}_patches.npy", patches)

    del patches, positions
    gc.collect()
    
    
    tqdm.write(f"  {filename[:40]}... : {n_patches} patches | Mem: {mem_gb:.1f} GB")
    

print(f"\nProcessed {len(sar_files)} files")

Pass 2 Extracting Patches...



Processing:   0%|          | 0/46 [00:00<?, ?it/s]

  s1a-iw-grd-vh-20251114t225109-20251114t2... : 12705 patches | Mem: 0.0 GB
  s1a-iw-grd-vv-20251114t225109-20251114t2... : 13782 patches | Mem: 0.6 GB
  s1a-iw-grd-vh-20260111t095455-20260111t0... : 9555 patches | Mem: 0.6 GB
  s1a-iw-grd-vv-20260111t095455-20260111t0... : 16124 patches | Mem: 0.6 GB
  s1a-iw-grd-vh-20260111t233937-20260111t2... : 24973 patches | Mem: 0.6 GB
  s1a-iw-grd-vv-20260111t233937-20260111t2... : 2573 patches | Mem: 0.6 GB
  s1a-iw-grd-vh-20260116t113541-20260116t1... : 17027 patches | Mem: 0.6 GB
  s1a-iw-grd-vv-20260116t113541-20260116t1... : 9247 patches | Mem: 0.6 GB
  s1a-iw-grd-vh-20260116t231538-20260116t2... : 25054 patches | Mem: 0.6 GB
  s1a-iw-grd-vv-20260116t231538-20260116t2... : 16855 patches | Mem: 0.6 GB
  s1a-iw-grd-vh-20260117t061452-20260117t0... : 24027 patches | Mem: 0.6 GB
  s1a-iw-grd-vv-20260117t061452-20260117t0... : 9641 patches | Mem: 1.1 GB
  s1a-iw-grd-vh-20260117t104141-20260117t1... : 12769 patches | Mem: 0.6 GB
  s1a-iw-grd-vv-

OSError: 1617166336 requested and 0 written

In [11]:
from pathlib import Path

output_dir = Path("patches")

# Check what's done
existing = list(output_dir.glob("*.npy"))
print(f"Files saved: {len(existing)}")

# Check disk space
import shutil
total, used, free = shutil.disk_usage("D:/")
print(f"Free space: {free / 1e9:.1f} GB")

Files saved: 0
Free space: 78.1 GB


In [12]:
existing = set(f.stem.replace("_patches", "") for f in outputdir.glob("*.npy"))
print(f"Already saved: {len(existing)} files")

# Find remaining files
remaining = [p for p in sar_files if Path(p).stem not in existing]
print(f"Remaining: {len(remaining)} files")

# Process only remaining files
for path in tqdm(remaining, desc="Processing"):
    filename = Path(path).stem
    
    mem_gb = psutil.Process().memory_info().rss / 1e9
    if mem_gb > 25:
        print(f"WARNING: Memory at {mem_gb:.1f} GB, stopping early")
        break
    
    image = load_sar_image(path)
    image_shape = image.shape
    
    normalised, params = preprocess_sar_complete(
        image, vmin=global_vmin, vmax=global_vmax
    )
    del image
    gc.collect()
    
    patches, positions = extract_patches(
        normalised, patch_size=patchsize, stride=stride, min_valid=min_valid
    )
    del normalised
    gc.collect()
    
    n_patches = len(patches)
    filestats.append({
        'filename': filename,
        'shape': image_shape,
        'patches': n_patches
    })
    
    if n_patches > 0:
        np.save(outputdir / f"{filename}_patches.npy", patches)
    
    del patches, positions
    gc.collect()
    
    tqdm.write(f"  {filename[:40]}... : {n_patches} patches | Mem: {mem_gb:.1f} GB")

print("Done!")

Already saved: 46 files
Remaining: 1 files


Processing:   0%|          | 0/1 [00:00<?, ?it/s]

  s1c-iw-grd-vv-20260117t225204-20260117t2... : 9436 patches | Mem: 7.9 GB
Done!


In [16]:
# Get all tiff filenames (stems)
tiff_stems = set(Path(p).stem for p in sar_files)

# Get all saved patch filenames (remove "_patches" suffix)
saved_stems = set(f.stem.replace("_patches", "") for f in outputdir.glob("*.npy"))

# Find missing
missing = tiff_stems - saved_stems

print(f"Total TIFFs: {len(tiff_stems)}")
print(f"Saved patches: {len(saved_stems)}")
print(f"Missing: {len(missing)}")

if missing:
    print("\nMissing files:")
    for m in sorted(missing):
        print(f"  {m}")

Total TIFFs: 44
Saved patches: 47
Missing: 0


In [19]:
import struct

def get_npy_shape(filepath):
    """Read shape from .npy header without loading data"""
    with open(filepath, 'rb') as f:
        # Skip magic number and version
        f.read(8)
        # Read header length
        header_len = struct.unpack('<H', f.read(2))[0]
        # Read header
        header = f.read(header_len).decode('latin1')
        # Parse shape from header string
        shape_start = header.find('(') + 1
        shape_end = header.find(')')
        shape_str = header[shape_start:shape_end]
        shape = tuple(int(x.strip()) for x in shape_str.split(',') if x.strip())
        return shape

total_patches = 0
for f in outputdir.glob("*_patches.npy"):
    shape = get_npy_shape(f)
    total_patches += shape[0]
    
print(f"Total patches: {total_patches:,}")

Total patches: 720,953


In [20]:
total_bytes = sum(f.stat().st_size for f in outputdir.glob("*_patches.npy"))
print(f"Total size: {total_bytes / 1e9:.1f} GB")

Total size: 182.5 GB


In [21]:
patch_files = list(outputdir.glob("*patches.npy"))

sample = np.load(patch_files[0], mmap_mode="r")
patch_shape = sample.shape[1:]
print(f"Patch shape: {patch_shape}")

Patch shape: (256, 256)


In [24]:
# Save metadata only
metadata = {
    'vmin': global_vmin,
    'vmax': global_vmax,
    'patch_size': patchsize,
    'stride': stride,
    'min_valid': min_valid,
    'num_patches': total_patches,
}
np.save(outputdir / 'metadata.npy', metadata, allow_pickle=True)

# Save shuffle index
shuffle_idx = np.random.permutation(total_patches)
np.save(outputdir / "shuffle_idx.npy", shuffle_idx)

In [27]:
# Delete the old combined file
(outputdir / "all_patches.npy").unlink()

# Re-run indexing (exclude all_patches.npy)
patch_files = sorted(f for f in outputdir.glob("*_patches.npy") if f.name != "all_patches.npy")

file_index = []
for f in tqdm(patch_files, desc="Indexing"):
    shape = get_npy_shape(f)
    file_index.append((f, shape[0]))

total_patches = sum(n for _, n in file_index)
print(f"Total patches: {total_patches:,}")

np.random.seed(42)
shuffle_idx = np.random.permutation(total_patches)
np.save(outputdir / "shuffle_idx.npy", shuffle_idx)

metadata = {
    'vmin': global_vmin,
    'vmax': global_vmax,
    'patch_size': patchsize,
    'stride': stride,
    'min_valid': min_valid,
    'num_patches': total_patches,
    'file_index': [(str(f), n) for f, n in file_index]
}
np.save(outputdir / 'metadata.npy', metadata, allow_pickle=True)

print("Done!")

Indexing:   0%|          | 0/44 [00:00<?, ?it/s]

Total patches: 720,953
Done!


In [28]:
# List all patch files and their sizes
for f in sorted(outputdir.glob("*_patches.npy")):
    shape = get_npy_shape(f)
    print(f"{f.name}: {shape[0]:,} patches")

s1a-iw-grd-vh-20251114t225109-20251114t225134-061880-07bcb4-002_patches.npy: 12,705 patches
s1a-iw-grd-vh-20260111t095455-20260111t095520-062718-07dd3c-002_patches.npy: 9,555 patches
s1a-iw-grd-vh-20260111t233937-20260111t234002-062726-07dd91-002_patches.npy: 24,973 patches
s1a-iw-grd-vh-20260116t113541-20260116t113606-062792-07e02d-002_patches.npy: 17,027 patches
s1a-iw-grd-vh-20260116t231538-20260116t231603-062799-07e070-002_patches.npy: 25,054 patches
s1a-iw-grd-vh-20260117t061452-20260117t061517-062803-07e08d-002_patches.npy: 24,027 patches
s1a-iw-grd-vh-20260117t104141-20260117t104206-062806-07e0a7-002_patches.npy: 12,769 patches
s1a-iw-grd-vh-20260117t104256-20260117t104321-062806-07e0a7-002_patches.npy: 25,117 patches
s1a-iw-grd-vh-20260117t122220-20260117t122245-062807-07e0ad-002_patches.npy: 19,989 patches
s1a-iw-grd-vh-20260117t125115-20260117t125140-062807-07e0b1-002_patches.npy: 9,304 patches
s1a-iw-grd-vh-20260117t154520-20260117t154545-062809-07e0c2-002_patches.npy: 18,17

In [31]:
# Cell 6: Verify
import matplotlib.pyplot as plt

# Load metadata
metadata = np.load(outputdir / 'metadata.npy', allow_pickle=True).item()
shuffle_idx = np.load(outputdir / "shuffle_idx.npy")

print(f"Total patches: {metadata['num_patches']:,}")
print(f"Global bounds: [{metadata['vmin']:.2f}, {metadata['vmax']:.2f}] dB")
print(f"Patch size: {metadata['patch_size']}")
print(f"Files: {len(metadata['file_index'])}")

# Load a few random patches for visualization
def load_random_patches(n=12):
    """Load n random patches from the dataset"""
    patches = []
    
    # Build cumsum for file lookup
    cumsum = [0]
    for _, count in metadata['file_index']:
        cumsum.append(cumsum[-1] + count)
    
    random_indices = np.random.choice(len(shuffle_idx), n, replace=False)
    
    # Group by file to minimize loads
    file_requests = {}
    for idx in random_indices:
        real_idx = shuffle_idx[idx]
        for i, (start, end) in enumerate(zip(cumsum[:-1], cumsum[1:])):
            if start <= real_idx < end:
                local_idx = real_idx - start
                if i not in file_requests:
                    file_requests[i] = []
                file_requests[i].append(local_idx)
                break
    
    # Load from each file
    for file_idx, local_indices in file_requests.items():
        fpath, _ = metadata['file_index'][file_idx]
        data = np.load(fpath)  # Full load, no mmap
        for local_idx in local_indices:
            patches.append(data[local_idx])
        del data
        gc.collect()
    
    return patches

# Show random samples
samples = load_random_patches(12)

fig, axes = plt.subplots(3, 4, figsize=(12, 9))
for ax, patch in zip(axes.flatten(), samples):
    ax.imshow(patch, cmap='gray', vmin=0, vmax=1)
    ax.axis('off')
plt.suptitle(f"Random samples from {metadata['num_patches']:,} patches")
plt.tight_layout()
plt.show()

Total patches: 720,953
Global bounds: [14.77, 24.54] dB
Patch size: 256
Files: 44


ValueError: Failed to read all data for array. Expected (24676, 256, 256) = 1617166336 elements, could only read 0 elements. (file seems not fully written?)